In [6]:
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import datetime
from io import StringIO
from new_strategy import Asset
from pathlib import Path
from new_strategy import Asset 


In [11]:
import pandas as pd

# === CONFIG ===
file_path = 'data/raw/XAUUSD/combined_data.csv.csv'   # Replace with your actual file path
timestamp_column = 'timestamp'  # Adjust if your timestamp column is named differently
expected_freq = '1min'         # Use '1min' or '1D' depending on your data

df = pd.read_csv(file_path)

# Combine 'Date' and 'Timestamp' into a single datetime
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Timestamp'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

# Drop the original Date and Timestamp columns (optional)
df.drop(columns=['Date', 'Timestamp'], inplace=True)

# === CHECK TIME RANGE ===
start = df.index.min()
end = df.index.max()
print(f"\n📅 Data time range: {start} to {end}")
print(f"📊 Number of rows: {len(df)}")

# === CHECK REQUIRED COLUMNS ===
required_cols = ['Open', 'High', 'Low', 'Close']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"❌ Missing required columns: {missing_cols}")
else:
    print("✅ All required OHLC columns are present.")

# === CHECK FOR MISSING TIMESTAMPS ===
full_index = pd.date_range(start=start, end=end, freq=expected_freq)
missing_timestamps = full_index.difference(df.index)

print(f"\n⏳ Total expected timestamps: {len(full_index)}")
print(f"❌ Missing timestamps: {len(missing_timestamps)}")

if not missing_timestamps.empty:
    print("🔍 Sample missing timestamps:", missing_timestamps[:10])

# Optional: Save missing timestamps to file
# pd.Series(missing_timestamps).to_csv("missing_timestamps.csv", index=False)

# === GROUP MISSING TIMESTAMPS BY DATE ===
missing_dates = pd.Series(missing_timestamps).dt.normalize()  # strip to date only
missing_day_counts = missing_dates.value_counts().sort_index()

# Add weekday info
missing_day_info = pd.DataFrame({
    'missing_count': missing_day_counts,
})
missing_day_info['weekday'] = missing_day_info.index.day_name()

# Group by weekday to see overall pattern
weekday_summary = missing_day_info.groupby('weekday')['missing_count'].sum().sort_values(ascending=False)

print("\n📆 Missing Timestamps by Day:")
print(missing_day_info.head(10))  # Show a few specific dates

print("\n📊 Total Missing Timestamps by Weekday:")
print(weekday_summary)

# Optional: Save to CSV for review
# missing_day_info.to_csv("missing_day_info.csv")




📅 Data time range: 2020-01-01 23:00:00 to 2024-12-31 21:58:00
📊 Number of rows: 1772913
✅ All required OHLC columns are present.

⏳ Total expected timestamps: 2629379
❌ Missing timestamps: 856466
🔍 Sample missing timestamps: DatetimeIndex(['2020-01-02 03:33:00', '2020-01-02 04:26:00',
               '2020-01-02 21:21:00', '2020-01-02 22:00:00',
               '2020-01-02 22:01:00', '2020-01-02 22:02:00',
               '2020-01-02 22:03:00', '2020-01-02 22:04:00',
               '2020-01-02 22:05:00', '2020-01-02 22:06:00'],
              dtype='datetime64[ns]', freq=None)

📆 Missing Timestamps by Day:
            missing_count    weekday
2020-01-02             63   Thursday
2020-01-03            120     Friday
2020-01-04           1440   Saturday
2020-01-05           1380     Sunday
2020-01-06             60     Monday
2020-01-07             60    Tuesday
2020-01-08             60  Wednesday
2020-01-09             60   Thursday
2020-01-10            120     Friday
2020-01-11         

In [7]:
ASSET = Asset.BTCUSD
asset_name = ASSET.value

# Load the already merged minute-level data
df = pd.read_csv(f"data/raw/{asset_name}/combined_data.csv", parse_dates=['timestamp'], index_col='timestamp')
df.index = df.index.tz_localize(None)
if 'ATR' in df.columns:
    df.drop(columns='ATR', inplace=True)

# === Ensure all days are present in index and forward-fill ===
start_date = df.index.min().normalize()
end_date = df.index.max().normalize()
full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Add missing days with last known values to preserve continuity for rolling metrics
df_daily_filled = df.resample('1D').agg({
    'high': 'max',
    'low': 'min',
    'close': 'last'
}).reindex(full_date_range)

df_daily_filled.ffill(inplace=True)

# Use forward-filled daily OHLC to preserve rolling consistency
daily_ohlc = df_daily_filled.copy()

#t10y
t10y = pd.read_csv("data/preprocessing_data/T10YIE.csv", parse_dates=['observation_date'], index_col='observation_date')
t10y.index = t10y.index.tz_localize(None)
t10y.rename(columns={t10y.columns[0]: 't10yie'}, inplace=True)
t10y = t10y.asfreq('D').ffill()
t10y['t10yie'] = t10y['t10yie'].shift(1)
daily_ohlc = daily_ohlc.merge(t10y, how='left', left_index=True, right_index=True)

#CPI
cpi = pd.read_csv("data/preprocessing_data/CPIAUCSL.csv", parse_dates=['observation_date'], index_col='observation_date')
cpi.index = cpi.index.tz_localize(None)
cpi.rename(columns={cpi.columns[0]: 'cpiaucsl'}, inplace=True)
cpi['cpiaucsl'] = cpi['cpiaucsl'].shift(1)
cpi = cpi.asfreq('D').ffill()
daily_ohlc = daily_ohlc.merge(cpi, how='left', left_index=True, right_index=True)

#VIX
vix = pd.read_csv("data/preprocessing_data/VIX_History.csv", parse_dates=['date'], index_col='date')
vix.index = vix.index.tz_localize(None)
vix['vix_close'] = vix['close'].shift(1)
vix = vix[['vix_close']]
daily_ohlc = daily_ohlc.merge(vix, how='left', left_index=True, right_index=True)

#DTW
dollar = pd.read_csv("data/preprocessing_data/DTWEXBGS.csv", parse_dates=['observation_date'], index_col='observation_date')
dollar.index = dollar.index.tz_localize(None)
dollar.rename(columns={dollar.columns[0]: 'dtwexbgs'}, inplace=True)
dollar['dtwexbgs'] = dollar['dtwexbgs'].shift(1)
dollar = dollar.asfreq('D').ffill()
daily_ohlc = daily_ohlc.merge(dollar, how='left', left_index=True, right_index=True)

#DGS10
dgs10 = pd.read_csv("data/preprocessing_data/DGS10.csv", parse_dates=['observation_date'], index_col='observation_date')
dgs10.index = dgs10.index.tz_localize(None)
dgs10.rename(columns={dgs10.columns[0]: 'dgs10'}, inplace=True)
dgs10['dgs10'] = dgs10['dgs10'].shift(1)
dgs10 = dgs10.asfreq('D').ffill()
daily_ohlc = daily_ohlc.merge(dgs10, how='left', left_index=True, right_index=True)

daily_ohlc['prev_close'] = daily_ohlc['close'].shift(1)
# Calculate True Range (TR)
daily_ohlc['true_range'] = daily_ohlc.apply(
    lambda row: max(
        row['high'] - row['low'],
        abs(row['high'] - row['prev_close']),
        abs(row['low'] - row['prev_close'])
    ) if pd.notnull(row['prev_close']) else None,
    axis=1
)

#Calculate 14-day ATR (excluding current day via shift)
daily_ohlc['atr_14'] = daily_ohlc['true_range'].rolling(window=14).mean().shift(1)

#Calculate moving averages on daily close (also shifted)
daily_ohlc['ma_14'] = daily_ohlc['close'].rolling(window=14).mean().shift(1)
daily_ohlc['ma_30'] = daily_ohlc['close'].rolling(window=30).mean().shift(1)
daily_ohlc['ma_100'] = daily_ohlc['close'].rolling(window=100).mean().shift(1)

# Rolling max/min of the daily close
daily_ohlc['max_14'] = daily_ohlc['close'].rolling(window=14).max().shift(1)
daily_ohlc['min_14'] = daily_ohlc['close'].rolling(window=14).min().shift(1)

daily_ohlc['max_30'] = daily_ohlc['close'].rolling(window=30).max().shift(1)
daily_ohlc['min_30'] = daily_ohlc['close'].rolling(window=30).min().shift(1)

daily_ohlc['max_100'] = daily_ohlc['close'].rolling(window=100).max().shift(1)
daily_ohlc['min_100'] = daily_ohlc['close'].rolling(window=100).min().shift(1)



# Cumulative max of close price
daily_ohlc['static_peak'] = daily_ohlc['close'].cummax()

# Static drawdown from that peak
daily_ohlc['drawdown_static'] = (
    (daily_ohlc['close'] - daily_ohlc['static_peak']) / daily_ohlc['static_peak']
).shift(1)

daily_ohlc['drawdown_30'] = (
    (daily_ohlc['close'] - daily_ohlc['close'].rolling(window=30).max()) 
    / daily_ohlc['close'].rolling(window=30).max()
).shift(1)

# Forward-fill daily values into minute-level data
df['daily_high'] = daily_ohlc['high'].reindex(df.index, method='ffill')
df['daily_low'] = daily_ohlc['low'].reindex(df.index, method='ffill')
df['daily_close'] = daily_ohlc['close'].reindex(df.index, method='ffill')
df['true_range'] = daily_ohlc['true_range'].reindex(df.index, method='ffill')
df['atr_14'] = daily_ohlc['atr_14'].reindex(df.index, method='ffill')
df['ma_14'] = daily_ohlc['ma_14'].reindex(df.index, method='ffill')
df['ma_30'] = daily_ohlc['ma_30'].reindex(df.index, method='ffill')
df['ma_100'] = daily_ohlc['ma_100'].reindex(df.index, method='ffill')

#Day and Weeknumber
df['day_of_week'] = df.index.dayofweek  # Monday=0, Sunday=6
df['week_number'] = df.index.isocalendar().week
df['hour_of_day'] = df.index.hour

df['max_price_14'] = daily_ohlc['max_14'].reindex(df.index, method='ffill')
df['min_price_14'] = daily_ohlc['min_14'].reindex(df.index, method='ffill')

df['max_price_30'] = daily_ohlc['max_30'].reindex(df.index, method='ffill')
df['min_price_30'] = daily_ohlc['min_30'].reindex(df.index, method='ffill')

df['max_price_100'] = daily_ohlc['max_100'].reindex(df.index, method='ffill')
df['min_price_100'] = daily_ohlc['min_100'].reindex(df.index, method='ffill')

#Drawdown
df['drawdown_static'] = daily_ohlc['drawdown_static'].reindex(df.index, method='ffill')
df['drawdown_30'] = daily_ohlc['drawdown_30'].reindex(df.index, method='ffill')

# Forward-fill T10YIE into minute-level data
df['t10yie'] = daily_ohlc['t10yie'].reindex(df.index, method='ffill')
#FF CPI
df['cpiaucsl'] = daily_ohlc['cpiaucsl'].reindex(df.index, method='ffill')
#FF vix
df['vix_close'] = daily_ohlc['vix_close'].reindex(df.index, method='ffill')
#FF Dollar 
df['dtwexbgs'] = daily_ohlc['dtwexbgs'].reindex(df.index, method='ffill')
#FF DGS10
df['dgs10'] = daily_ohlc['dgs10'].reindex(df.index, method='ffill')

# 1. Compute HMM Features on Daily Data
daily_ohlc['return_raw'] = daily_ohlc['close'].pct_change()
daily_ohlc['volatility_raw'] = (
    daily_ohlc['close']
    .rolling(window=10)
    .apply(lambda x: np.mean((x - x.mean())**2))
).shift(1)

# === Shifted versions for use in model (to avoid lookahead bias) ===
daily_ohlc['return'] = daily_ohlc['return_raw'].shift(1)
daily_ohlc['volatility'] = daily_ohlc['volatility_raw'].shift(1)

df['daily_return'] = daily_ohlc['return'].reindex(df.index, method='ffill')
df['daily_volatility'] = daily_ohlc['volatility'].reindex(df.index, method='ffill')

# 2. Drop NaNs (due to rolling/shift)
hmm_features = daily_ohlc[['return_raw', 'volatility_raw']].dropna()

# 3. Fit HMM on ALL data (no slicing)
model = GaussianHMM(n_components=3, covariance_type='full', n_iter=75, random_state=42)
model.fit(hmm_features)
hmm_features['regime'] = model.predict(hmm_features)

# 4. Label Regimes (Bull, Bear, Neutral) based on return means
regime_means = hmm_features.groupby('regime')['return_raw'].mean().sort_values()
regime_mapping = {
    regime_means.index[0]: 'Bear',
    regime_means.index[1]: 'Neutral',
    regime_means.index[2]: 'Bull'
}
hmm_features['regime_label'] = hmm_features['regime'].map(regime_mapping)

# 5. Merge regime info into daily_ohlc
daily_ohlc['regime'] = hmm_features['regime']
daily_ohlc['regime_label'] = hmm_features['regime_label']

# 6. Forward-fill daily regime into minute-level df
df['regime'] = daily_ohlc['regime'].reindex(df.index, method='ffill')
df['regime_label'] = daily_ohlc['regime_label'].reindex(df.index, method='ffill')

#calc avg 30 day return
daily_ohlc['avg_return_30d'] = daily_ohlc['return_raw'].rolling(window=30).mean().shift(1)
df['avg_return_30d'] = daily_ohlc['avg_return_30d'].reindex(df.index, method='ffill')
#Optional: Inspect regime distribution
print(df['regime_label'].value_counts())

# Overwrite the original file
df.index = df.index.tz_localize('UTC')
output_path = f"data/processed/{asset_name}/combined_data.csv"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path)
print(f"✅ Saved processed data to: {output_path}")


Bear       879656
Neutral    833390
Bull       814182
Name: regime_label, dtype: int64
✅ Saved processed data to: data/processed/BTCUSD/combined_data.csv


In [8]:
"""import pandas as pd

# === Settings ===
asset_name = "WTI"  # or "WTI"
path = f"data/raw/{asset_name}/combined_data.csv"

# === Load the data ===
df = pd.read_csv(path, parse_dates=['timestamp'], index_col='timestamp')
df.index = df.index.tz_localize(None)

# === Determine date range ===
start_date = df.index.min().normalize()
end_date = df.index.max().normalize()
full_range = pd.date_range(start=start_date, end=end_date, freq='D')

# === Actual calendar days in the data ===
actual_days = df.index.normalize().unique()
actual_days = pd.DatetimeIndex(actual_days)

# === Find missing days ===
missing_days = full_range.difference(actual_days)

# === Output ===
print(f"📆 Start date: {start_date.date()}")
print(f"📆 End date:   {end_date.date()}")
print(f"🗓️ Total calendar days: {len(full_range)}")
print(f"📉 Missing days: {len(missing_days)}\n")

if len(missing_days) > 0:
    print("🚫 Missing dates and their weekdays:")
    for day in missing_days:
        print(f" - {day.date()} ({day.strftime('%A')})")
else:
    print("✅ No missing dates in raw data.")"""


'import pandas as pd\n\n# === Settings ===\nasset_name = "WTI"  # or "WTI"\npath = f"data/raw/{asset_name}/combined_data.csv"\n\n# === Load the data ===\ndf = pd.read_csv(path, parse_dates=[\'timestamp\'], index_col=\'timestamp\')\ndf.index = df.index.tz_localize(None)\n\n# === Determine date range ===\nstart_date = df.index.min().normalize()\nend_date = df.index.max().normalize()\nfull_range = pd.date_range(start=start_date, end=end_date, freq=\'D\')\n\n# === Actual calendar days in the data ===\nactual_days = df.index.normalize().unique()\nactual_days = pd.DatetimeIndex(actual_days)\n\n# === Find missing days ===\nmissing_days = full_range.difference(actual_days)\n\n# === Output ===\nprint(f"📆 Start date: {start_date.date()}")\nprint(f"📆 End date:   {end_date.date()}")\nprint(f"🗓️ Total calendar days: {len(full_range)}")\nprint(f"📉 Missing days: {len(missing_days)}\n")\n\nif len(missing_days) > 0:\n    print("🚫 Missing dates and their weekdays:")\n    for day in missing_days:\n      